In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler


url = "https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data"
df = pd.read_csv(url)

# Drop the non-predictive ID column and separate features/target
X = df.drop(columns=['name', 'status'])
y = df['status']

# Standardize features: TabNet can work without, but scaling often helps convergence
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [8]:
from sklearn.model_selection import train_test_split


X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.30, random_state=42, stratify=y
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print(f"Train size: {X_train.shape[0]}, Valid size: {X_valid.shape[0]}, Test size: {X_test.shape[0]}")


Train size: 136, Valid size: 29, Test size: 30


In [9]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

clf = TabNetClassifier()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


/home/rudra/.local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [10]:
import optuna
from sklearn.metrics import accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier  

def objective(trial):
    params = {
        'n_d': trial.suggest_categorical('n_d', [8, 16, 32]),
        'n_a': trial.suggest_categorical('n_a', [8, 16, 32]),
        'n_steps': trial.suggest_int('n_steps', 3, 5),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'lambda_sparse': trial.suggest_float('lambda_sparse', 1e-5, 1e-3, log=True),  # Updated
        'optimizer_fn': torch.optim.Adam,
        'optimizer_params': dict(lr=trial.suggest_float('lr', 1e-3, 1e-1, log=True))  # Updated
    }
    model = TabNetClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric=['accuracy'],
        max_epochs=50,
        patience=10,
        batch_size=32,
        virtual_batch_size=16,
    )
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)
print("Best trial:", study.best_trial.params)



[I 2025-05-09 05:07:02,723] A new study created in memory with name: no-name-2f5f0d54-1cbb-47c9-a3d5-63ad0d83454d


epoch 0  | loss: 1.38674 | val_0_accuracy: 0.68966 |  0:00:00s
epoch 1  | loss: 0.62443 | val_0_accuracy: 0.75862 |  0:00:00s
epoch 2  | loss: 0.50818 | val_0_accuracy: 0.75862 |  0:00:00s
epoch 3  | loss: 0.45771 | val_0_accuracy: 0.68966 |  0:00:00s
epoch 4  | loss: 0.38803 | val_0_accuracy: 0.68966 |  0:00:00s
epoch 5  | loss: 0.47349 | val_0_accuracy: 0.72414 |  0:00:00s
epoch 6  | loss: 0.38094 | val_0_accuracy: 0.75862 |  0:00:00s
epoch 7  | loss: 0.39202 | val_0_accuracy: 0.75862 |  0:00:00s
epoch 8  | loss: 0.40343 | val_0_accuracy: 0.82759 |  0:00:00s
epoch 9  | loss: 0.37735 | val_0_accuracy: 0.7931  |  0:00:01s
epoch 10 | loss: 0.37723 | val_0_accuracy: 0.75862 |  0:00:01s
epoch 11 | loss: 0.34428 | val_0_accuracy: 0.7931  |  0:00:01s
epoch 12 | loss: 0.31854 | val_0_accuracy: 0.72414 |  0:00:01s
epoch 13 | loss: 0.35084 | val_0_accuracy: 0.75862 |  0:00:01s
epoch 14 | loss: 0.31227 | val_0_accuracy: 0.86207 |  0:00:01s
epoch 15 | loss: 0.30635 | val_0_accuracy: 0.93103 |  0

/home/rudra/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-05-09 05:07:05,552] Trial 0 finished with value: 0.9310344827586207 and parameters: {'n_d': 32, 'n_a': 8, 'n_steps': 3, 'gamma': 1.1739585789048637, 'lambda_sparse': 0.0002412938365734335, 'lr': 0.013228654467939298}. Best is trial 0 with value: 0.9310344827586207.


epoch 25 | loss: 0.22771 | val_0_accuracy: 0.82759 |  0:00:02s

Early stopping occurred at epoch 25 with best_epoch = 15 and best_val_0_accuracy = 0.93103
epoch 0  | loss: 0.7003  | val_0_accuracy: 0.62069 |  0:00:00s


/home/rudra/.local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

epoch 10 | loss: 0.43816 | val_0_accuracy: 0.75862 |  0:00:01s

Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_0_accuracy = 0.82759


/home/rudra/.local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.63127 | val_0_accuracy: 0.7931  |  0:00:00s
epoch 1  | loss: 0.74196 | val_0_accuracy: 0.68966 |  0:00:00s
epoch 2  | loss: 0.48808 | val_0_accuracy: 0.72414 |  0:00:00s
epoch 3  | loss: 0.49748 | val_0_accuracy: 0.75862 |  0:00:00s
epoch 4  | loss: 0.43976 | val_0_accuracy: 0.68966 |  0:00:00s
epoch 5  | loss: 0.39148 | val_0_accuracy: 0.72414 |  0:00:00s
epoch 6  | loss: 0.42729 | val_0_accuracy: 0.7931  |  0:00:00s
epoch 7  | loss: 0.43495 | val_0_accuracy: 0.82759 |  0:00:01s
epoch 8  | loss: 0.36267 | val_0_accuracy: 0.75862 |  0:00:01s
epoch 9  | loss: 0.3925  | val_0_accuracy: 0.82759 |  0:00:01s
epoch 10 | loss: 0.3367  | val_0_accuracy: 0.75862 |  0:00:01s
epoch 11 | loss: 0.35376 | val_0_accuracy: 0.68966 |  0:00:01s
epoch 12 | loss: 0.39369 | val_0_accuracy: 0.7931  |  0:00:01s
epoch 13 | loss: 0.3432  | val_0_accuracy: 0.68966 |  0:00:01s
epoch 14 | loss: 0.32886 | val_0_accuracy: 0.75862 |  0:00:01s
epoch 15 | loss: 0.33595 | val_0_accuracy: 0.75862 |  0

/home/rudra/.local/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-05-09 05:07:57,328] Trial 19 finished with value: 0.9655172413793104 and parameters: {'n_d': 16, 'n_a': 16, 'n_steps': 3, 'gamma': 1.3358767750235476, 'lambda_sparse': 0.00012079651778824189, 'lr': 0.05879131688961723}. Best is trial 2 with value: 0.9655172413793104.


epoch 35 | loss: 0.27276 | val_0_accuracy: 0.96552 |  0:00:04s

Early stopping occurred at epoch 35 with best_epoch = 25 and best_val_0_accuracy = 0.96552
Best trial: {'n_d': 8, 'n_a': 8, 'n_steps': 3, 'gamma': 1.9168452073142168, 'lambda_sparse': 1.3648047351037876e-05, 'lr': 0.06466701589324549}
